In [ ]:
import itertools
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib
matplotlib.use('TkAgg')  
import matplotlib.pyplot as plt
import easygui as eg
import numpy as np
from ipywidgets import interact
from scipy import stats
from scipy.stats import f_oneway
import matplotlib.ticker as mticker
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
df = pd.read_csv('output.csv')

In [ ]:
# Create unique IDs for Artist and Album (Daan)
df['Artist_ID'] = pd.factorize(df['Artist'])[0] +1
df['Album_ID'] = pd.factorize(df['Album'])[0] + 1

df_album = df[['Artist_ID', 'Album', 'Metascore', 'Releasedate', 'Album_ID', 'Recordlabel']]

df_cr = df[['CriticNames', 'CriticScores', 'CriticReviewDates', 'CriticReviewTexts']]

# Corrected initialization of new_df (Jochem)
new_df = pd.DataFrame(columns=['CriticNames', 'CriticScores', 'CriticReviewDates', 'CriticReviewTexts'])

# Create a list to store the split values
split_values_list = []

# Iterate through each column in the original DataFrame (Jayshree)
for col in df_cr.columns:
    # Split the strings in each column by '@' and stack them into a single column
    split_values = df_cr[col].str.split('@', expand=True).stack()
    
    # Reset the index and keep the first level as a separate column
    split_values = split_values.reset_index(level=0, drop=True).rename('original_index')
    
    # Append the split values to the list
    split_values_list.append(split_values)

# Concatenate the split values outside the loop (bart)
new_df = pd.concat(split_values_list, axis=1)

# Add the 'OriginalIndex' column to the concatenated DataFrame (bart)
new_df['OriginalIndex'] = new_df.index

# Reset the entire index to make it continuous (Bart)
new_df = new_df.reset_index(drop=True)

# Rename the columns to match the original DataFrame's column names (Jochem)
new_df.columns = df_cr.columns.tolist() + ['OriginalIndex']
new_df['Album_ID'] = (new_df['OriginalIndex'] == 0).cumsum()
new_df = new_df.drop('OriginalIndex', axis=1, errors='ignore')
df_reviews = new_df.copy()
df_reviews['cr_ID'] = pd.factorize(df_reviews['CriticReviewTexts'])[0] + 1

# Extract unique genres from the 'Genre' column of the CSV file (daan)
unique_genres = df['Genre'].str.split(',', expand=True).stack().unique()

# Create a DataFrame for the unique genres with a unique ID for each genre (Jayshree)
Genre_list = pd.DataFrame(unique_genres, columns=['Genre'])
Genre_list['Genre_ID'] = range(1, len(Genre_list) + 1)

# Recreate the genre_rel DataFrame (jayshree)
genre_rel_data = []
for index, row in df.iterrows():
    album_id = row['Album_ID']
    genres = row['Genre'].split(',')
    for genre in genres:
        genre_id = Genre_list[Genre_list['Genre'] == genre]['Genre_ID'].iloc[0]
        genre_rel_data.append({'Album_ID': album_id, 'Genre_ID': genre_id})
genre_rel_df = pd.DataFrame(genre_rel_data)
genre_rel_df['Dummy_ID2'] = range(1, len(genre_rel_df) + 1)

# Calculate 'Number_albums' which is the count of albums per artist. (daan)
number_albums = df.groupby('Artist')['Album'].nunique().reset_index(name='Number_albums')

# Calculate 'First_album' which is the earliest release date per artist. (Jochem)
first_album = df.groupby('Artist')['Releasedate'].min().reset_index(name='First_album')

# Merge these calculations with the original artist DataFrame (Jochem)
df_artist = df[['Artist']].drop_duplicates()
df_artist = df_artist.merge(number_albums, left_on='Artist', right_on='Artist', how='left')
df_artist = df_artist.merge(first_album, left_on='Artist', right_on='Artist', how='left')

# Rename the columns to match the ERD (Bart)
df_artist.columns = ['Name_artist', 'Number_albums', 'First_album']

# Add the 'Artist_ID' from the previously factorized ID
df_artist['Artist_ID'] = pd.factorize(df_artist['Name_artist'])[0]

# The final DataFrame should have the columns in the order: Artist_ID, Name_artist, Number_albums, First_album (Bart)
df_artist = df_artist[['Artist_ID', 'Name_artist', 'Number_albums', 'First_album']]

# Check if 'Artist_ID' and 'Album_ID' columns are in 'df' (Daan)
if 'Artist_ID' not in df.columns:
    df['Artist_ID'] = pd.factorize(df['Artist'])[0]
if 'Album_ID' not in df.columns:
    df['Album_ID'] = pd.factorize(df['Album'])[0]

# Now, we'll create the 'Artist_rel' DataFrame which relates artists with albums.
# We'll use a dummy primary key 'Dummy_ID' for this relationship table. (Jayshree)
df_artist_rel = df[['Artist_ID', 'Album_ID']].drop_duplicates().reset_index(drop=True)
df_artist_rel['Dummy_ID1'] = df_artist_rel.index + 1  # Start 'Dummy_ID' at 1 for database convention

# Ordering the columns according to the screenshot provided (Daan)
df_artist_rel = df_artist_rel[['Dummy_ID1', 'Artist_ID', 'Album_ID']]




In [ ]:
# This is done by Daan van der Veldt

# Create a function to show the input dialog
def get_db_connection_details():
    msg = "Enter Database Connection Details"
    title = "Database Connection"
    field_names = ["Host", "Database Name", "Username", "Password"]
    default_values = ["localhost", "postgres", "myuser", "mypassword"]

    field_values = eg.multenterbox(msg, title, field_names, default_values)

    # Check if the user canceled the input dialog
    if field_values is None:
        return None

    return {
        "host": field_values[0],
        "database": field_values[1],
        "user": field_values[2],
        "password": field_values[3]
    }

# Get database connection details from the user
db_details = get_db_connection_details()

if db_details is not None:
    # Create a new connection
    connection = psycopg2.connect(
        host=db_details["host"],
        database=db_details["database"],
        user=db_details["user"],
        password=db_details["password"]
    )

# Create cursor
cursor = connection.cursor()

# Get a list of all table names in the current schema
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")

table_names = [table[0] for table in cursor.fetchall()]

for table_name in table_names:
    cursor.execute(f"DROP TABLE IF EXISTS {table_name} CASCADE")

connection.commit()

# Create a SQLAlchemy engine using the obtained connection details
engine = create_engine(f"postgresql://{db_details['user']}:{db_details['password']}@{db_details['host']}/{db_details['database']}")


In [ ]:
# This is done by Jayshree

# Import the artist relation data into SQL
df_artist_rel.to_sql('artists_rel', engine, if_exists='replace', index=False)

# Import the album data into SQL
df_album.to_sql('album_list', engine, if_exists='replace', index=False)

# Import the review data into SQL
df_reviews.to_sql('Critic_reviews', engine, if_exists='replace', index=False)

# Import the genre list data into SQL
Genre_list.to_sql('genre_list', engine, if_exists='replace', index=False)

# Import the relations of genres data into SQL
genre_rel_df.to_sql('genre_rel', engine, if_exists='replace', index=False)

# Import the artist data into SQL
df_artist.to_sql('artist_list', engine, if_exists='replace', index=False)

In [ ]:
# Create a cursor object
cursor = connection.cursor()

sql_command = """
    ALTER TABLE artists_rel 
    ADD CONSTRAINT "Dummy_ID1" PRIMARY KEY ("Dummy_ID1");

    ALTER TABLE album_list 
    ADD CONSTRAINT "Album_ID" PRIMARY KEY ("Album_ID");

    ALTER TABLE artist_list 
    ADD CONSTRAINT "Artist_ID" PRIMARY KEY ("Artist_ID");

    ALTER TABLE "Critic_reviews" 
    ADD CONSTRAINT "cr_ID" PRIMARY KEY ("cr_ID");

    ALTER TABLE genre_rel 
    ADD CONSTRAINT "Dummy_ID2" PRIMARY KEY ("Dummy_ID2");

    ALTER TABLE "genre_list" 
    ADD CONSTRAINT "Genre_ID" PRIMARY KEY ("Genre_ID");
"""
# Execute the SQL command
cursor.execute(sql_command)

# Commit the changes to the database
connection.commit()


In [ ]:
# This is done by Jochem
# Create a cursor object
cursor = connection.cursor()

sql_command = """
    ALTER TABLE "artist_list"
    ALTER COLUMN "Name_artist"
    SET DATA TYPE VARCHAR(50); 

    ALTER TABLE "artist_list"
    ALTER COLUMN "Number_albums"
    SET DATA TYPE INT;

    ALTER TABLE "artist_list"
    ALTER COLUMN "First_album"
    SET DATA TYPE DATE USING "First_album"::date;

    ALTER TABLE "album_list"
    ALTER COLUMN "Album"
    SET DATA TYPE VARCHAR(100); 

    ALTER TABLE "album_list"
    ALTER COLUMN "Metascore"
    SET DATA TYPE INT;

    ALTER TABLE "album_list"
    ALTER COLUMN "Recordlabel"
    SET DATA TYPE VARCHAR(50);
    
    ALTER TABLE "Critic_reviews"
    ALTER COLUMN "CriticNames"
    SET DATA TYPE VARCHAR(50); 

    ALTER TABLE "Critic_reviews"
    ALTER COLUMN "CriticReviewTexts"
    SET DATA TYPE VARCHAR(500); 

    ALTER TABLE "genre_list"
    ALTER COLUMN "Genre"
    SET DATA TYPE VARCHAR(50); 
"""

# Execute the SQL command
cursor.execute(sql_command)

# Commit the changes to the database
connection.commit()